<a href="https://colab.research.google.com/github/zzeeddd95/capstone-project/blob/master/SVR%2Blinearregmsft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import pandas_datareader as web


/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [ ]:
#Retrieve stock data
msft = web.DataReader('MSFT', data_source='yahoo', start='2010-05-27', end='2020-07-06')
print(msft.head())

                 High    Low       Open      Close       Volume  Adj Close
Date                                                                      
2010-05-27  26.360001  25.73  25.730000  26.000000  136433600.0  20.539593
2010-05-28  26.120001  25.66  25.840000  25.799999   67496900.0  20.381601
2010-06-01  26.309999  25.52  25.530001  25.889999   76152400.0  20.452698
2010-06-02  26.480000  25.73  26.059999  26.459999   65718800.0  20.902985
2010-06-03  26.930000  26.41  26.549999  26.860001   67837000.0  21.218981


In [ ]:
#Filter out data to only obtain close price (independant variable)
msft = msft[['Close']]
msft

,Close
Date,
2010-05-27,26.000000
2010-05-28,25.799999
2010-06-01,25.889999
2010-06-02,26.459999
2010-06-03,26.860001
...,...
2020-06-29,198.440002
2020-06-30,203.509995
2020-07-01,204.699997


In [ ]:
#Create a variable to allow for predicting 'n days' into the future
future_out =30

In [ ]:
#Create the dependant variable which will be shifted 'n' units up
msft['Prediction'] = msft[['Close']].shift(-future_out)
msft.tail()

,Close,Prediction
Date,,
2020-06-29,198.440002,NaN
2020-06-30,203.509995,NaN
2020-07-01,204.699997,NaN
2020-07-02,206.259995,NaN
2020-07-06,210.699997,NaN


In [ ]:
#create independant data set by converting the dataframe to a numpy array
msftdf = np.array(msft.drop(['Prediction'],1))
#Renove the last 'n' rows
msftdfx = msftdf[:-future_out] #therefore for every column, all of the rows will displayed - future out
msftdfx

array([[ 26.        ],
       [ 25.79999924],
       [ 25.88999939],
       ...,
       [183.63000488],
       [185.66000366],
       [183.42999268]])

In [ ]:
#creat dependant dataset (prediction), all values including any Nan's
msftdfy = np.array(msft['Prediction'])
#Filter this data by obtaining all values except the last 'n' rows
msftdfy = msftdfy[:-future_out]
msftdfy

array([ 24.82999992,  25.12999916,  25.44000053, ..., 204.69999695,
       206.25999451, 210.69999695])

In [ ]:
#split data into 80% training and 20%testing
msftdfx_train , msftdfx_test, msftdfy_train, msftdfy_test = train_test_split(msftdfx, msftdfy, test_size = 0.2)

In [ ]:
#Create and train the SVR model
SVRmsft = SVR(kernel='rbf', C=1e3,gamma=0.1)
SVRmsft.fit(msftdfx_train, msftdfy_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
#test model accuracy via score which returns the coefficient of determination (R^2) of prediction
SVMconfidence = SVRmsft.score(msftdfx_test, msftdfy_test)
print("SVM confidence", SVMconfidence)

SVM confidence 0.9838390933915906


In [ ]:
#Create and train the linear regression model
linear = LinearRegression()
#train the model
linear.fit(msftdfx_train, msftdfy_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
#test model accuracy via score which returns the coefficient of determination (R^2) of prediction
linearconfidence = linear.score(msftdfx_test, msftdfy_test)
print("Linear regression confidence", linearconfidence)

Linear regression confidence 0.9856496544636397


In [ ]:
#set msft_forecast = to the last 30 rows of original dataset from close price column
msft_forecast = np.array(msft.drop(['Prediction'],1))[-future_out:]
print(msft_forecast)


[[183.50999451]
 [181.57000732]
 [181.80999756]
 [181.3999939 ]
 [183.25      ]
 [182.83000183]
 [184.91000366]
 [185.36000061]
 [182.91999817]
 [187.19999695]
 [188.36000061]
 [189.80000305]
 [196.83999634]
 [186.27000427]
 [187.74000549]
 [188.94000244]
 [193.57000732]
 [194.24000549]
 [196.32000732]
 [195.1499939 ]
 [200.57000732]
 [201.91000366]
 [197.83999634]
 [200.33999634]
 [196.33000183]
 [198.44000244]
 [203.50999451]
 [204.69999695]
 [206.25999451]
 [210.69999695]]


In [ ]:
#Print the SVR predicted values for the upcoming 'n' days
SVRprediction = SVRmsft.predict(msft_forecast)
print(SVRprediction)

[171.23981745 204.42421205 205.08873147 203.09978603 178.51977434
 190.27679264 155.84817441 160.45483335 187.8724322  158.35802501
 119.75189684  85.15546033 107.88624691 169.03118616 142.06516909
 101.11676977 104.79712001 106.51216395 107.84953745 107.51151991
 107.9100549  107.9100604  107.90668903 107.9100503  107.85058041
 107.909117   107.91006054 107.91006054 107.91006054 107.91006054]


In [ ]:
#Print the linear regression predicted values for the upcoming 'n' days
linearprediction = linear.predict(msft_forecast)
print(linearprediction)

[191.25443671 189.22292973 189.47424162 189.04489584 190.98217684
 190.54236506 192.72049194 193.19171771 190.63660702 195.11851672
 196.3332441  197.84117932 205.21328755 194.14465121 195.68400042
 196.94060778 201.78903529 202.49064093 204.66876781 203.44355846
 209.11926353 210.52247481 206.26046301 208.87840167 204.67923381
 206.88877468 212.19794596 213.44408732 215.07767849 219.7271401 ]


In [ ]:
#Retrieve stock data
msftstock = web.DataReader('MSFT', data_source='yahoo', start='2020-05-28', end='2020-07-05')
msftstock = msftstock[['Close']]
msftstock.head()

,Close
Date,
2020-05-28,181.399994
2020-05-29,183.250000
2020-06-01,182.830002
2020-06-02,184.910004
2020-06-03,185.360001
